# WEKA Proiektua
Hau da gure WEKA-ko proiektua, Natural lenguage processing-era bideratutakoa. Proiektu hau egiteko, hurrengo pausuak jarraituko ditugu. Gomendatzen da dokumentazioaren aldi berean irakurtzea.


## Indizea
* [Sarrera: Proiektuarena aurkezpena](#sarrera)
* [Datuak](#datuak)
* [Data pre-processing](#dpr)
* [Exploratory Data Analysis](#exploratory)
* [Modeloaren sorrera](#model)
* [Emaitzak eta eztabaida](#emaitzak)
* [Model deployment](#deployment)
* [Konklusioa](#conclusion)

## Sarrera: Proiektuarena aurkezpena <a name="sarrera"></a>